# Priemtesten en factorisatie

**Naam**: Ferre Van der Vieren (r0851455)

## Trial division

De meest eenvoudig te begrijpen manier om een getal te factoriseren in priemfactoren is trial division. Implementeeer de methode $\texttt{trial_division(n)}$ die een lijst teruggeeft met priemfactoren van $n$. Bij trial division moet je enkel delen door priemgetallen. De methode $\texttt{find_primes(m)}$ geeft dus een lijst van alle priemgetallen kleiner dan $m$ terug. Indien je dit niet vindt, probeer dan trial division te implementeren zonder een lijst van priemgetallen te gebruiken.

In [76]:
import math
def find_primes(m):
    primes = []
    
    for num in range(2, m):
        for divisor in range(2, num):
            if (num % divisor) == 0:
                break
        else:
            primes.append(num)
    return primes

def trial_division(n):
    primefacs = []
    
    primes = find_primes(n) # 'every composite number m is divisible by a number d <= sqrt(m)'
    num = n
    i = 0
    while i < len(primes):
        if (num % primes[i] != 0):
            i += 1
        else:
            primefacs.append(primes[i])
            num = int(num / primes[i])
            if (num == 1): # all primefacs found
                break

            i = 0  # restart loop
    return primefacs

## Schor's algoritme

Hieronder implementeren we stapsgewijs het algoritme van Schor voor het factoriseren van een getal $m$ in twee priemfactoren $p$ en $q$ zodat $m = p\cdot q$. Op het vinden van de orde van een element na is elke stap eigenlijk al eerder in een andere oefening geimplementeerd. Indien je vorige computeroefeningen gedaan hebt kan je deze dus gewoon kopiëren. 

In [77]:
import random
import decimal

def gcd(a, m):
    return eaa(a, m, False)

# using Extended Euclidean Algorithm to efficiently calculate gcd or Bezout's identity
# @return
    #  if (bezout == False): gcd(a, b)
    #  if (bezout == True): Bezout's identity coefficients s and t for which gcd(a, b) = s*a + t*b
def eaa(x, y, bezout = True):
    (a, new_a) = (x, y)
    (b, new_b) = (1, 0)
    (c, new_c) = (0, 1)
    
    while new_a != 0:
        q = a // new_a
        
        # coefficients
        (a, new_a) = (new_a, a - q * new_a)
        (b, new_b) = (new_b, b - q * new_b)
        (c, new_c) = (new_c, c - q * new_c)
        
    if (bezout):
        return (b, c)
    return a

def order(a, m):
    if (gcd(a,m) != 1):
        return -1
    result = 1
    i = 1
    
    for i in range(1,m):
        result = (result * a) % m
        if (result == 1):
            return i

# @pre
    # m must be of the form m = pq with p,q primes
# @return
    # prime factorization of m (tuple of p and q) using Shor's algorithm
def schor(m):
    e = 0
    a = 0
    while (e % 2 == 1 or pow(a, int(e/2), m) == 1 or pow(a, int(e/2), m) == (m-1)):  # (-1 mod m) is (m-1 mod m) in Python; % always positive!
        a = random.randint(1, m)
        g = gcd(a, m)
        if (g != 1):
            return g, int((m / g))
        e = order(a, m)
        
    p = gcd(m, (pow(a, int(e/2), m) - 1) % m)  # (a – b) % c = ((a % c) – (b % c)) % c
    q = gcd(m, (pow(a, int(e/2), m) + 1) % m)  # (a + b) % c = ((a % c) + (b % c)) % c
    
    return p, q

Test op het getal 8683 om te kijken of je implementatie werkt.

<u> Antwoord:</u>

In [78]:
schor(8683)  # 8683 = 19 * 457

(19, 457)

## Stelling van Boneh

Het bewijs van de stelling van Boneh uit de cursus is constructief en biedt dus een manier om $m = p\cdot q$ te factoriseren. Implementeer een methode $\texttt{boneh(m, e, d, n)}$ die met een hoge waarschijnlijkheid $p$ en $q$ vindt, gegeven het getal $m$, de encryptie en decryptiesleutel $e, d$ en een gekozen aantal willekeurige getallen $n$.

In [79]:
def boneh(m, e, d, n):
    k = e*d - 1
    
    g = 0
    temp = k
    r = 0
    while ((r % 2) == 0):  # k = (2^g)r for r odd
        g += 1
        r = (k / pow(2, g))
    r = int(r)     # needed for later on
    
    i = 0
    while i < n:
        w = random.randint(1, m-1) 
        if (gcd(w, m) != 1):    # w is invertible in Z_m (and thus a unit) <=> gcd(w, m) = 1
            i -= 1              # w is no unit, thus this does not count as an iteration
            continue
        for f in range(1, g+1): 
            c = pow(w, int(pow(2, f-1)*r), m)
            if (c != 1 and c != (m-1) and pow(c, 2, m) == 1):
                return gcd(c+1, m), gcd(c-1, m)
        i += 1
        
    return -1   # did not find factorisation (chance is less than 1/(2^n))

In [80]:
boneh(8683, 5, 4925, 100)

(457, 19)

Vergelijk nu de snelheid van de implementaties. Gebruik hiervoor [timeit](https://docs.python.org/2/library/timeit.html). 

In [75]:
import timeit

%timeit trial_division(8683)       # obviously the slowest
%timeit schor(8683)                # much, much more efficient than trial_division
%timeit boneh(8683, 5, 4925, 100)  # slightly faster runtime than schor()


262 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
77.8 µs ± 7.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15.3 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
